In [6]:
import geopandas as gpd
import numpy as np
from functools import reduce

In [8]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWgzetye0t_tlnZGwEDLuhIAItfc4vpnBuuLuza2YLuQp8AjHENANbY



Successfully saved authorization token.


In [9]:
def feature2ee(file):
    try:
        # check if the file is shapefile or CSV
        if file.endswith('.shp'):
            gdf = gpd.read_file(file, encoding='utf-8')
            g = [i for i in gdf.geometry] # rows of polygon in the shp
            features = []
            
            #for Polygon geo data type
            if (gdf.geom_type[0] == 'Polygon'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry] # do we need this again?
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()
                    
                    g = ee.Geometry.Polygon(cords)
                    feature = ee.Feature(g)
                    features.append(feature)
                print('done')
                
                ee_object = ee.FeatureCollection(features)
                
                return ee_object
            
            # for LineString geo data type
            elif (gdf.geom_type[0] == 'LineString'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()
                    double_list = reduce(lambda x,y: x+y, cords)
                    
                    g = ee.Geometry.LineString(double_list)
                    feature = ee.Feature(g)
                    features.append(feature)
                print('done')
                
                ee_object = ee.FeatureCollection(features)
                
                return ee_object
            
            # for Point geo data type
            elif (gdf.geom_type[0] == 'Point'):
                for i in range(len(g)):
                    g = [i for i in gdf.geometry]
                    x,y = g[i].exterior.coords.xy
                    cords = np.dstack((x,y)).tolist()
                    double_list = reduce(lambda x,y: x+y, cords)
                    single_list = reduce(lambda x,y: x+y, double_list)
                    
                    g = ee.Geometry.Point(single_list)
                    feature = ee.Feature(g)
                    features.append(feature)
                print('done')
                
                ee_object = ee.FeatureCollection(features)
                
                return ee_object
         # check if the file is CSV
        elif file.endswith('.csv'):
            df = pd.read_csv(file)
            features = []
            for i in range(df.shape[0]):
                x,y = df.x[i], df.y[i]
                latlon = [x,y]
                g = ee.Geometry.Point(latlong)
                feature = ee.Feature(g)
                features.append(feature)
            print('done')
            
            ee_object = ee.FeatureCollection(features)
            return ee_object
    except:
        print('Wrong file format, or lacking of neccesary package. Make sure you have ee, numpy, geopandas imported.')
                
                

In [10]:
zam_dist = '/media/sitian/Samsung_T5/District_boundaries_and_yields/ZAM_Districts_2010/ZAM_Districts.shp'
zam_dist_ee = feature2ee(zam_dist)

done


In [12]:
zam_dist_ee.propertyNames().getInfo()

KeyboardInterrupt: 

In [13]:
zam_on_ee = ee.FeatureCollection('users/stxiong8/zambia_all_boundaries/Zam_national_boundary_epsg4326')

In [14]:
zam_on_ee.propertyNames().getInfo()

['system:version', 'system:id', 'system:asset_size']